In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./item.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Description_Url'].tolist()

总数量：25


['https://vi.vipr.ebaydesc.com/itmdesc/273142816010?t=1565807490000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1',
 'https://vi.vipr.ebaydesc.com/itmdesc/273996486803?t=1575486695000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1',
 'https://vi.vipr.ebaydesc.com/itmdesc/274193643518?t=0&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1',
 'https://vi.vipr.ebaydesc.com/itmdesc/274316783418?t=0&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1',
 'https://vi.vipr.ebaydesc.com/itmdesc/274316790657?t=1585067498000&category=50454&seller=zamo-zuan&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secu

In [3]:
crawler_status = 'error'

print(crawler_status)

error


In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [5]:
a = 0

resp = requests.get(input_.loc[a, 'Description_Url'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'<!doctype html><html lang=zh><head><link rel="dns-prefetch" href="//ir.ebaystatic.com"><link rel="dns-prefetch" href="//secureir.ebaystatic.com"><link rel="dns-prefetch" href="//i.ebayimg.com"><link rel="dns-prefetch" href="//rover.ebay.com"><script>$ssgST=new Date().getTime();</script><link rel=preload fetchpriority=high href=https://ir.ebaystatic.com/cr/v/c1/market-sans/v1.0/MarketSans-Regular-WebS.woff2 as=font type=font/woff2><style nonce=/T9lRWCiFL42lfYV2Ho2Jw==>\n            body { font-family: "Market Sans", Arial, sans-serif; }\n        </style><meta charset=UTF-8><meta name=viewport content="width=device-width, initial-scale=1"><title>eBay</title><link rel="stylesheet" href="https://ir.ebaystatic.com/rs/c/item-desc-2Vk6qq44.css"></head><body><noscript id="afph0"></noscript><script src="https://ir.ebaystatic.com/rs/c/item-desc-wp-NMN7j.js"></script>\n<script nonce="/T9lRWCiFL42lfYV2Ho2Jw==">$_mod.ready();</script> <script nonce="/T9lRWCiFL42lfYV2Ho2Jw==" src="https://ir.ebayst

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

In [7]:
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./description.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html lang="zh">
 <head>
  <link href="//ir.ebaystatic.com" rel="dns-prefetch"/>
  <link href="//secureir.ebaystatic.com" rel="dns-prefetch"/>
  <link href="//i.ebayimg.com" rel="dns-prefetch"/>
  <link href="//rover.ebay.com" rel="dns-prefetch"/>
  <script>
   $ssgST=new Date().getTime();
  </script>
  <link as="font" fetchpriority="high" href="https://ir.ebaystatic.com/cr/v/c1/market-sans/v1.0/MarketSans-Regular-WebS.woff2" rel="preload" type="font/woff2"/>
  <style nonce="/T9lRWCiFL42lfYV2Ho2Jw==">
   body { font-family: "Market Sans", Arial, sans-serif; }
  </style>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   eBay
  </title>
  <link href="https://ir.ebaystatic.com/rs/c/item-desc-2Vk6qq44.css" rel="stylesheet"/>
 </head>
 <body>
  <noscript id="afph0">
  </noscript>
  <script src="https://ir.ebaystatic.com/rs/c/item-desc-wp-NMN7j.js">
  </script>
  <script nonce="/T9lRWCiFL42lfYV2Ho2Jw==">
   $_mod.re

In [8]:
list_p = html.xpath('//div[@id="pagewrap"]/p[not(@class)]')

list_p

[<Element p at 0x121d11800>,
 <Element p at 0x1217a6d80>,
 <Element p at 0x121a25880>,
 <Element p at 0x121a27b00>]

In [9]:
list_ul = html.xpath('//div[@id="pagewrap"]/ul')

list_ul

[<Element ul at 0x121a25fc0>,
 <Element ul at 0x121a240c0>,
 <Element ul at 0x121a24040>,
 <Element ul at 0x121a25d00>]

In [10]:
len(list_p) == len(list_ul)

True

In [12]:
df_temp = pd.DataFrame([{'Description_Url': input_.loc[a, 'Description_Url']}])

for p, ul in zip(list_p, list_ul):
    name = p.xpath('./*/text()')[0].strip()
    name = name[:-1].strip() if name.endswith(':') else name
    df_temp.loc[0, name] = '\n'.join([li.strip() for li in ul.xpath('./li/text()')])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

IndexError: list index out of range

In [ ]:
crawler_status = 'ok'

crawler_status

In [ ]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Description_Url': input_.loc[a, 'Description_Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

In [ ]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Description_Url'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_description.xlsx', index=False)

output_correct

In [ ]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Description_Url'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_description_error.xlsx', index=False)

output_error